In [20]:
import torch
from tqdm import tqdm
import random

In [21]:
class Node:
    def __init__(self, data, parents, children = None) -> None:
        self.data = data
        self.visits = 0
        self.score = 0
        self.parents = parents
        if children == None:
            self.children = []
        else:
            self.children = children
    def find_node(self, data, eq):
        if eq(data, self.data):
            return [self]
        finds = []
        for child in self.children:
            finds.extend(child.find_node(data, eq))
        return finds
    def find_leafs(self):
        if len(self.children) == 0:
            return [self]
        finds = []
        for child in self.children:
            finds.extend(child.find_leafs())
        return finds
    def cout_nodes(self):
        return 1 + sum([c.cout_nodes() for c in self.children])
    def listify(self):
        found = []
        for c in self.children:
            found.extend(c.listify())

        return [self.data] + found
    

In [22]:
def make_children(tensor, current_peice):
    children = []
    for i in range(tensor.size(0)):
        for j in range(tensor.size(1)):
            if tensor[i][j] == 0:
                # Create a copy of the original tensor
                new_tensor = tensor.clone()
                # Replace 0 with 1 at this index
                new_tensor[i][j] = current_peice
                # Append the modified tensor to the list
                children.append(new_tensor)
    return children

In [23]:
def winning_state_pice(tensor, own_pice):
    suums = torch.cat((torch.sum(tensor, dim = 0), torch.sum(tensor, dim = 1)))
    a = torch.any(torch.eq(suums, tensor.shape[0] * own_pice)) 
    return a

In [24]:
def winning_state(tensor):
    a = winning_state_pice(tensor, -1) or winning_state_pice(tensor, 1)
    return a

In [25]:
torch.equal(torch.zeros((3,3)), torch.zeros((3,3)))

True

In [26]:
graph = Node(torch.zeros((3,3)), None)  
frontier = [graph]
current_peice = 1
progress_bar = tqdm(total=518000, miniters=1000)

i = 0
while True:
    new_frontier = []
    for f in frontier:
        i += 1
        if i % 1000 == 0:
            progress_bar.update(1000)
            i = 0
        
        children = [Node(d, f) for d in make_children(f.data, current_peice)]
        new_frontier.extend([c for c in children if not winning_state(c.data)])
        f.children = children

    frontier = new_frontier
    current_peice *= -1
    if len(frontier) == 0:
        break
    
# Close the progress bar
progress_bar.close()

100%|██████████| 518000/518000 [02:43<00:00, 3165.56it/s]


In [29]:
len(graph.listify()), graph.cout_nodes()

(672346, 672346)

In [10]:
def score_leaf(node, own_peice):
    if winning_state_pice(node.data,own_peice):
        score = 1
    elif winning_state_pice(node.data,own_peice * -1):
        score = -1
    else:
        score = 0
    node.score += score
    node.visits += 1
    parent = node.parent
    while parent != None:
        parent.score += score
        parent.visits += 1
        parent = parent.parent

In [11]:
own_peice = 1
for leaf in graph.find_leafs():
    score_leaf(leaf, own_peice)
graph.visits

283608

In [12]:
def get_best_child(graph : Node, state):
    node = graph.find_node(state, torch.equal)[0]
    scored_states = {n.data: n.score / n.visits for n in node.children}
    best_states =  [key for key, value in scored_states.items() if value == max(scored_states.values())]
    best_random =  random.choice(best_states)
    return best_random
get_best_child(graph, graph.data)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 1., 0.]])